In [43]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [44]:

jobs_csv_path = os.path.join(
    "..", "data", "business-establishments-and-jobs-data-by-business-size-and-industry.csv")

jobs_data = pd.read_csv(jobs_csv_path)
jobs_data.head()

,Census year,CLUE small area,ANZSIC indusrty,CLUE industry,Business size,Total establishments,Total jobs
0,2007,West Melbourne (Residential),"Transport, Postal and Warehousing","Transport, Postal and Storage",Small business,20,132.0
1,2006,Carlton,Accommodation and Food Services,Accommodation,Small business,17,102.0
2,2006,Carlton,Administrative and Support Services,Admin and Support Services,Non employing,1,NaN
3,2006,Carlton,Arts and Recreation Services,Arts and Recreation Services,Medium business,4,207.0
4,2006,Carlton,Construction,Construction,Medium business,5,196.0


In [45]:
jobs_data.columns

Index(['Census year', 'CLUE small area', 'ANZSIC indusrty', 'CLUE industry',
       'Business size', 'Total establishments', 'Total jobs'],
      dtype='object')

In [46]:
jobs_data = jobs_data.groupby(["Census year",'ANZSIC indusrty','CLUE small area', "Total establishments", "Total jobs"]).size().reset_index(name= "counts")
jobs_data

,Census year,ANZSIC indusrty,CLUE small area,Total establishments,Total jobs,counts
0,2002,Accommodation and Food Services,Carlton,3,0.0,1
1,2002,Accommodation and Food Services,Carlton,4,139.0,1
2,2002,Accommodation and Food Services,Carlton,7,0.0,1
3,2002,Accommodation and Food Services,Carlton,11,327.0,1
4,2002,Accommodation and Food Services,Carlton,16,106.0,1
...,...,...,...,...,...,...
10356,2022,Wholesale Trade,Southbank,3,222.0,1
10357,2022,Wholesale Trade,Southbank,9,69.0,1
10358,2022,Wholesale Trade,West Melbourne (Industrial),6,299.0,1
10359,2022,Wholesale Trade,West Melbourne (Industrial),24,151.0,1


In [50]:
jobs_data_y = jobs_data
wanted_year = [2022]
wanted_job = ["---"]
jobs_data_idy = jobs_data_y["Census year"].isin(wanted_year)

jobs_data_y = jobs_data_y[jobs_data_idy]


jobs_data_y = jobs_data_y.groupby(["Census year","CLUE small area",'ANZSIC indusrty', "Total jobs"]).size().reset_index(name= "counts")
summarized_y = jobs_data_y.groupby(["CLUE small area",'ANZSIC indusrty'])['Total jobs'].sum().reset_index()
jobs_data_y
summarized_y
jobs_data_y

,Census year,CLUE small area,ANZSIC indusrty,Total jobs,counts
0,2022,Carlton,Accommodation and Food Services,0.0,1
1,2022,Carlton,Accommodation and Food Services,189.0,1
2,2022,Carlton,Accommodation and Food Services,255.0,1
3,2022,Carlton,Accommodation and Food Services,682.0,1
4,2022,Carlton,Accommodation and Food Services,1227.0,1
...,...,...,...,...,...
488,2022,West Melbourne (Residential),"Rental, Hiring and Real Estate Services",31.0,1
489,2022,West Melbourne (Residential),Retail Trade,78.0,1
490,2022,West Melbourne (Residential),Retail Trade,95.0,1
491,2022,West Melbourne (Residential),"Transport, Postal and Warehousing",51.0,1


In [51]:
summarized_y

,CLUE small area,ANZSIC indusrty,Total jobs
0,Carlton,Accommodation and Food Services,2353.0
1,Carlton,Administrative and Support Services,68.0
2,Carlton,Arts and Recreation Services,327.0
3,Carlton,Construction,52.0
4,Carlton,Education and Training,3531.0
...,...,...,...
202,West Melbourne (Residential),Public Administration and Safety,28.0
203,West Melbourne (Residential),"Rental, Hiring and Real Estate Services",31.0
204,West Melbourne (Residential),Retail Trade,173.0
205,West Melbourne (Residential),"Transport, Postal and Warehousing",51.0


In [52]:
jobs_data_y = jobs_data
wanted_year = [2022]
wanted_job = ["---"]
jobs_data_idy = jobs_data_y["Census year"].isin(wanted_year)
jobs_data_y = jobs_data_y[jobs_data_idy]
jobs_data_y

,Census year,ANZSIC indusrty,CLUE small area,Total establishments,Total jobs,counts
9866,2022,Accommodation and Food Services,Carlton,6,255.0,1
9867,2022,Accommodation and Food Services,Carlton,23,682.0,1
9868,2022,Accommodation and Food Services,Carlton,36,189.0,1
9869,2022,Accommodation and Food Services,Carlton,39,0.0,1
9870,2022,Accommodation and Food Services,Carlton,194,1227.0,1
...,...,...,...,...,...,...
10356,2022,Wholesale Trade,Southbank,3,222.0,1
10357,2022,Wholesale Trade,Southbank,9,69.0,1
10358,2022,Wholesale Trade,West Melbourne (Industrial),6,299.0,1
10359,2022,Wholesale Trade,West Melbourne (Industrial),24,151.0,1


In [53]:
#Now make a nice bokeh plot 

import pandas as pd
from squarify import normalize_sizes, squarify
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap

#Sum up total jobs
jobs_by_area_industry = jobs_data_y.groupby(["CLUE small area", "ANZSIC indusrty"]).sum("Total jobs")
jobs_by_area_industry = jobs_by_area_industry.reset_index()
print(jobs_by_area_industry.columns)

# Grouping by "industry" and summing "total jobs"
jobs_by_industry = jobs_by_area_industry.groupby("ANZSIC indusrty").sum("Total jobs")
jobs_by_industry = jobs_by_industry.reset_index()

# Function to generate treemap blocks
def treemap(jobs_data_y, col, x, y, dx, dy, *, N=100):
 if dx != 0 and dy != 0:
    sub_df = jobs_data_y.nlargest(N, col)
    normed = normalize_sizes(sub_df[col], dx, dy)
    blocks = squarify(normed, x, y, dx, dy)
    blocks_df = pd.DataFrame.from_dict(blocks).set_index(sub_df.index)
    return sub_df.join(blocks_df, how='left').reset_index()
 else:
    return pd.DataFrame()

x, y, w, h = 0, 0, 800, 450

# Generating blocks for treemap
blocks_by_industry = treemap(jobs_by_industry, "Total jobs", x, y, w, h)

# Generating blocks for each area within the industry
default_x = 1
default_y = 1
default_dx = 1
default_dy = 1
dfs = []
for index, row in blocks_by_industry.iterrows():
    if dx != 0 and dy != 0:
        industry = row["ANZSIC indusrty"]
        total_jobs = row["Total jobs"]
        x = row["x"]
        y = row["y"]
        dx = row["dx"]
        dy = row["dy"]
        if dx == 0:
           dx = default_dx
        if dy == 0:
           dy = default_dy
        if x == 0:
            x = default_x
        if y == 0:
            y = default_y
        sub_df = jobs_by_area_industry[jobs_by_area_industry["ANZSIC indusrty"] == industry]
        print(sub_df)
        sub_df = sub_df[sub_df["Total jobs"] != 0]
        dfs.append(treemap(sub_df, "Total jobs", x, y, dx, dy, N=10))
        blocks = pd.concat(dfs)
    else:
        print("failed")

# Bokeh plot setup
p = figure(width=w, height=h, tooltips="@CLUE small area", toolbar_location=None,
           x_axis_location=None, y_axis_location=None)
p.x_range.range_padding = p.y_range.range_padding = 0
p.grid.grid_line_color = None

# Rendering treemap blocks
p.rect('x', 'y', 'dx', 'dy', source=blocks, line_width=1, line_color="white",
       fill_alpha=0.8, fill_color="blue")

# Adding text for industry names
p.text('x', 'y', x_offset=2, text="ANZSIC indusrty", source=blocks_by_industry,
       text_font_size="18pt", text_color="white")

# Adding text for area names
blocks["ytop"] = blocks.y + blocks.dy
p.text('x', 'ytop', x_offset=2, y_offset=2, text="CLUE small area", source=blocks,
       text_font_size="6pt", text_baseline="top", text_color="black")

# Show the plot
show(p)

Index(['CLUE small area', 'ANZSIC indusrty', 'Census year',
       'Total establishments', 'Total jobs', 'counts'],
      dtype='object')
              CLUE small area ANZSIC indusrty  Census year  \
19  City of Melbourne (total)      All ANZSIC         8088   

    Total establishments  Total jobs  counts  
19                 13828    480576.0       4  
                  CLUE small area  \
11                        Carlton   
30      City of Melbourne (total)   
47                      Docklands   
63                 East Melbourne   
78                     Kensington   
95                Melbourne (CBD)   
110         Melbourne (Remainder)   
125               North Melbourne   
138                     Parkville   
151                Port Melbourne   
174                     Southbank   
201  West Melbourne (Residential)   

                                     ANZSIC indusrty  Census year  \
11   Professional, Scientific and Technical Services         4044   
30   Professional, Scie

In [54]:
import pandas as pd

from bokeh.core.properties import value
from bokeh.plotting import ColumnDataSource, figure, show
from bokeh.sampledata.sample_superstore import data as df
from bokeh.transform import cumsum, factor_cmap

rows = pd.crosstab(df.Category, df.Region, aggfunc='sum', values=df.Sales, normalize="index")

source = ColumnDataSource(rows.T)

cats = ["Office Supplies", "Furniture", "Technology"]
regions = source.data["Region"]

p = figure(y_range=cats, x_range=(-0.55, 1.02), height=400, width=700, tools="",
           x_axis_location=None, toolbar_location=None, outline_line_color=None)
p.grid.grid_line_color = None
p.yaxis.fixed_location = 0
p.axis.major_tick_line_color = None
p.axis.major_label_text_color = None
p.axis.axis_line_color = "#4a4a4a"
p.axis.axis_line_width = 6

source.data["color"] = [ "#dadada","#dadada", "#4a4a4a", "#dadada"]
for y in cats:
    left, right = cumsum(y, include_zero=True), cumsum(y)

    p.hbar(y=value(y), left=left, right=right, source=source, height=0.9,
           color=factor_cmap("Region", "MediumContrast4", regions))

    pcts = source.data[y]
    source.data[f"{y} text"] = [f"{r}\n{x*100:0.1f}%" for r, x in zip(regions, pcts)]

    p.text(y=value(y), x=left, text=f"{y} text", source=source, x_offset=10,
           text_color="color", text_baseline="middle", text_font_size="15px")

totals = pd.crosstab(df.Category, df.Region, margins=True, aggfunc='sum',
                     values=df.Sales, normalize="columns").All

p.hbar(right=0, left=-totals, y=totals.index, height=0.9, color="#dadada")

text = [f"{name} ({totals.loc[name]*100:0.1f}%)" for name in cats]
p.text(y=cats, x=0, text=text, text_baseline="middle", text_align="right",
       x_offset=-12, text_color="#4a4a4a", text_font_size="20px",
       text_font_style="bold")

show(p)

RuntimeError: bokeh sample data directory does not exist, please execute bokeh.sampledata.download()

In [ ]:
jobs_data_z = jobs_data
wanted_year = [2022]
wanted_job = ["---"]
jobs_data_idz = jobs_data_z["Census year"].isin(wanted_year)

jobs_data_z = jobs_data_z[jobs_data_idz]
jobs_data_z

summarized_z = jobs_data_z.groupby(['CLUE small area', 'ANZSIC indusrty']).agg({'Total jobs': 'sum'}).reset_index()
summarized_z

,CLUE small area,ANZSIC indusrty,Total jobs
0,Carlton,Accommodation and Food Services,2353.0
1,Carlton,Administrative and Support Services,68.0
2,Carlton,Arts and Recreation Services,327.0
3,Carlton,Construction,52.0
4,Carlton,Education and Training,3531.0
...,...,...,...
202,West Melbourne (Residential),Public Administration and Safety,28.0
203,West Melbourne (Residential),"Rental, Hiring and Real Estate Services",31.0
204,West Melbourne (Residential),Retail Trade,173.0
205,West Melbourne (Residential),"Transport, Postal and Warehousing",51.0


In [61]:

summarized_z["ANZSIC indusrty"]
summarized_z = summarized_z[summarized_z["ANZSIC indusrty"] != "All ANZSIC"]
summarized_z.reset_index(drop=True, inplace=True)


In [59]:
import pandas as pd
from bokeh.core.properties import value
from bokeh.plotting import ColumnDataSource, figure, show
from bokeh.transform import cumsum, factor_cmap

# Assuming you have already created the summarized dataframe with unique industries
# and their corresponding total jobs
# summarized_df = ...

summarized_z = summarized_z.drop_duplicates(subset=['ANZSIC indusrty'])

source = ColumnDataSource(summarized_z)

industries = summarized_z["ANZSIC indusrty"]
areas = summarized_z["CLUE small area"]
total_jobs = summarized_z["Total jobs"]

p = figure(y_range=industries, x_range=(0, max(total_jobs) * 1.1), height=400, width=700, tools="",
           x_axis_location=None, toolbar_location=None, outline_line_color=None)
p.grid.grid_line_color = None
p.yaxis.fixed_location = 0
p.axis.major_tick_line_color = None
p.axis.major_label_text_color = None
p.axis.axis_line_color = "#4a4a4a"
p.axis.axis_line_width = 6

# Assigning colors to industries
industry_colors = {"Accommodation and Food Services": "orchid",
    "Administrative and Support Services": "firebrick",
    "Agriculture, Forestry and Fishing": "skyblue",
    "Arts and Recreation Services":"lightcoral",
    "Construction":"gold",
    "Education and Training": "mediumpurple",
    "Electricity, Gas, Water and Waste Services": "olive",
    "Financial and Insurance Services": "darkorange",
    "Health Care and Social Assistance":"cadetblue",
    "Information Media and Telecommunications":"salmon",
    "Manufacturing": "lightseagreen",
    "Other Services": "skyblue",
    "Mining":"darkslateblue",
    "Professional, Scientific and Technical Services": "khaki",
    "Public Administration and Safety": "palegreen",
    "Rental, Hiring and Real Estate Services": "sienna",
    "Retail Trade": "lightskyblue",
    "Wholesale Trade": "thistle",
     "Mining":"darkslateblue",
    "Transport, Postal and Warehousing": "lightsteelblue",
                   }
#colors = ["#3182bd", "#6baed6", "#9ecae1", "#c6dbef", "#e6550d", "#fd8d3c", "#fdae6b", "#fdd0a2"]
source.data['color'] = [industry_colors[industry] for industry in industries]

# Plotting horizontal bars for total jobs
p.hbar(y='ANZSIC indusrty', right='Total jobs', height=0.9, source=source,
       color='color')

# Adding text labels for total jobs
p.text(y='ANZSIC indusrty', x='Total jobs', text='Total jobs', source=source,
       x_offset=5, text_baseline="middle", text_font_size="15px", text_color="black")

show(p)